# Midterm

The midterm project will consist of a comparison between several CNN architectures for tumor segmentation. The goal is both to create a high-performing algorithm for the target task, as well as to analyze performance across several different architecture permutations. In total, three different network designs will be tested. As each model is built and trained, ensure to serialize the final model `*.hdf5` file before moving to the next iteration.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model files for all three models
* final compiled `*.csv` file with performance statistics across the different architectures
* final 1-page write-up with methods and results of experiments

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [ ]:
# --- Install jarvis (only in Google Colab or local runtime)
!pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [ ]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets, custom

# Data

As in the tutorial, data for this assignment will consist of tumor MRI exams. The following lines of code will download the dataset (if not already present). Since the algorithms below may require slightly different model inputs, the required generators and inputs will be defined dyanically in the code blocks later in this notebook.

In [ ]:
# --- Download dataset
datasets.download(name='mr/brats-2020-mip')

{'code': '/data/raw/mr_brats_2020', 'data': '/data/raw/mr_brats_2020'}

# Training

A total of three different network architectures will be tested. The goal is to compare the incremental benefit of several design choices. After building and training each model to convergence, do not forget to save each model as a separate `*.hdf5` file.

## 1. 2D U-Net

In this algorithm a standard 2D U-Net architecture will be used to perform tumor segmentation. This network is **identical** to the baseline architectures presented in the week 5 and week 6 tutorials. The algorithm input will include an entire full field-of-view `240 x 240` resolution 2D slice from a brain MRI across 4 different modalities (channels). Key customizations to the standard U-Net architecture that should be implemented (as in the week 5 and week 6 tutorials) include:

* same padding (vs. valid padding)
* strided convolutions (vs. max-pooling)

### Create generators and inputs

In [ ]:

configs = {'specs': {'ys': {'tumor': {'norms': {'clip': {'max': 1}}}}}}
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-mip', keyword='mip*vox', configs=configs)
inputs = client.get_inputs(Input)

### Define model

In [ ]:
# 3x3 FILTERS | BATCH NORM | ReLU | STRITED CONV
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same',
    'kernel_initializer': 'he_normal'}
 
# --- Define block components
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
 
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)
 
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))
 
# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
 
# --- Define single transpose
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
 
# --- Define transpose block
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))
 
# --- Define expanding layers
l6 = tran2(48, l5)
 
l7 = tran2(32, conv1(48, l6+conv1(48, l4)))
l8 = tran2(16, conv1(32, l7+conv1(32, conv1(32, l3))))
l9 = tran2(8,  conv1(16, l8+conv1(16, conv1(16, conv1(16, l2)))))
l10 = conv1(8, conv1(8, l9+conv1(8, conv1(8, conv1(8, l1)))))
# --- Create logits
logits = {}
logits['tumor'] = layers.Conv3D(filters=2, name='tumor', **kwargs)(l10)
 
# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile and train model

In [ ]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.005),
    loss={'tumor': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={'tumor': custom.dsc(cls=1)},
    experimental_run_tf_function=False)

In [ ]:
model.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=9,
    validation_data=gen_valid,
    validation_steps=500,
    validation_freq=3,
    use_multiprocessing=True)

Epoch 1/9
500/500 [==============================] - 197s 361ms/step - loss: 0.0758 - dsc_1: 0.5432
Epoch 2/9
500/500 [==============================] - 183s 366ms/step - loss: 0.0188 - dsc_1: 0.7089
Epoch 3/9
500/500 [==============================] - 338s 676ms/step - loss: 0.0137 - dsc_1: 0.7811 - val_loss: 0.0132 - val_dsc_1: 0.8064
Epoch 4/9
500/500 [==============================] - 181s 362ms/step - loss: 0.0125 - dsc_1: 0.7863
Epoch 5/9
500/500 [==============================] - 182s 364ms/step - loss: 0.0109 - dsc_1: 0.8007
Epoch 6/9
500/500 [==============================] - 338s 676ms/step - loss: 0.0112 - dsc_1: 0.8141 - val_loss: 0.0129 - val_dsc_1: 0.8165
Epoch 7/9
500/500 [==============================] - 182s 365ms/step - loss: 0.0104 - dsc_1: 0.8207
Epoch 8/9
500/500 [==============================] - 184s 367ms/step - loss: 0.0101 - dsc_1: 0.8239
Epoch 9/9
500/500 [==============================] - 340s 680ms/step - loss: 0.0105 - dsc_1: 0.8261 - val_loss: 0.0106 - v

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, None, 240, 2 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, None, 240, 24 296         dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, 240, 24 32          conv3d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, None, 240, 24 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
def dice(y_true, y_pred, c=1, epsilon=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      (np.ndarray) y_true : ground-truth label
      (np.ndarray) y_pred : predicted logits scores
      (int)             c : class to calculate DSC on
    
    """
    assert y_true.ndim == y_pred.ndim
    
    true = y_true[..., 0] == c
    pred = np.argmax(y_pred, axis=-1) == c 

    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred) + epsilon
    
    return A / B


In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc_valid = []
dsc_train=[]

for x, y in test_train:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['tumor']

    # --- Argmax
    dsc_train.append(dice(y['tumor'][0], logits[0], c=1))

dsc_train = np.array(dsc_train)

for x, y in test_valid:
   # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['tumor']

    # --- Argmax
    dsc_valid.append(dice(y['tumor'][0], logits[0], c=1))
dsc_valid = np.array(dsc_valid)

[ 2021-05-14 16:53:01 ] [====================] 100.000% : Iterating | 000074    

In [ ]:
print(np.mean(dsc_train), np.mean(dsc_valid), np.median(dsc_valid), np.median(dsc_train), np.quantile(dsc_valid, 0.75), np.quantile(dsc_train, 0.75))



0.8302406021197961 0.7980558647745607 0.884943291022225 0.8788957909323272 0.9127488337803554 0.9156923394696403


In [ ]:
# --- Define columns
df_train = pd.DataFrame(index=np.arange(dsc_train.size))
df_valid=pd.DataFrame(index=np.arange(dsc_valid.size))
df_stats=pd.DataFrame(index=np.arange(1))

df_train['Dice score Training']=dsc_train
df_valid['Dice score Validation']=dsc_valid

df_stats['Dice score Training Median'] = df_train['Dice score Training'].median()
df_stats['Dice score Training Mean'] = df_train['Dice score Training'].mean()
df_stats['Dice score Training 25th percentile: '] = df_train['Dice score Training'].quantile(0.25)
df_stats['Dice score Training 75th percentile: '] = df_train['Dice score Training'].quantile(0.75)

df_stats['Dice score Validation Median'] = df_valid['Dice score Validation'].median()
df_stats['Dice score Validation Mean'] = df_valid['Dice score Validation'].mean()
df_stats['Dice score Validation 25th percentile: '] = df_valid['Dice score Validation'].quantile(0.25)
df_stats['Dice score Validation 75th percentile: '] = df_valid['Dice score Validation'].quantile(0.75)

# --- Print accuracy

In [ ]:
df_train.to_csv('./train_results_m1.csv')
df_valid.to_csv('./valid_results_m1.csv')
df_stats.to_csv('./results_summary.csv')
model.save('./model_1.hdf5')

In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Hybrid 3D/2D U-Net

In this algorithm, the original 2D model is modified to yield a hybrid 3D/2D model that uses several continguous slices for prediction of any given single output slice. The network is **identical** to the architecture presented in the week 6 tutorials. To ensure a fair comparison, recommend using similar network design and training hyperparameters as in the first 2D only model above. 

### Create generators and inputs

In [ ]:
# --- Input ==> 3 x 240 x 240 x 4
configs = {
    'specs': {
        'xs': {'dat': {'shape': [3, 240, 240, 4]}},
        'ys': {'tumor': {'norms': {'clip': {'max': 1}}}}}}

gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-mip', keyword='mip*vox', configs=configs)
inputs = client.get_inputs(Input)

In [ ]:
# --- Define 2D conv (xy-features)
conv_2d = lambda x, filters, strides : layers.Conv3D(
    filters=filters, 
    strides=strides, 
    kernel_size=(1, 3, 3), 
    padding='same',
    kernel_initializer='he_normal')(x)

# --- Define 1D conv (z-features)
conv_1d = lambda x, filters, k=2 : layers.Conv3D(
    filters=filters,
    strides=1,
    kernel_size=(k, 1, 1),
    padding='valid',
    kernel_initializer='he_normal')(x)

In [ ]:
# --- Define lambda functions
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)

# --- Define stride-1 3D, stride-2 3D and stride-1 1D (z-subsample) blocks
conv1 = lambda filters, x : relu(norm(conv_2d(x, filters, strides=(1, 1, 1))))
conv2 = lambda filters, x : relu(norm(conv_2d(x, filters, strides=(1, 2, 2))))
convZ = lambda filters, k, x : relu(norm(conv_1d(x, filters, k=k)))

# --- Define arbitrary input
dat = Input(shape=(3, 240, 240, 4))

# --- Define contracting layers
l1 = conv1(8,  inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, convZ(48, 2, conv2(48, l3)))
l5 = conv1(64, convZ(64, 2, conv2(64, l4)))


In [ ]:
Model(inputs=inputs, outputs=l5)({'dat': dat})

# --- 3-slices to 1-slice
p3 = convZ(32, 3, l3)
print(Model(inputs=inputs, outputs=l3)({'dat': dat}).shape)
print(Model(inputs=inputs, outputs=p3)({'dat': dat}).shape)

# --- 2-slices to 1-slice
p4 = convZ(48, 2, l4)
print(Model(inputs=inputs, outputs=l4)({'dat': dat}).shape)
print(Model(inputs=inputs, outputs=p4)({'dat': dat}).shape)

(None, 3, 60, 60, 32)
(None, 1, 60, 60, 32)
(None, 2, 30, 30, 48)
(None, 1, 30, 30, 48)


In [ ]:
# --- Define 2D transpose
tran = lambda x, filters : layers.Conv3DTranspose(
    filters=filters, 
    strides=(1, 2, 2),
    kernel_size=(1, 3, 3),
    padding='same',
    kernel_initializer='he_normal')(x)

# --- Define transpose block
tran2 = lambda filters, x : relu(norm(tran(x, filters)))

In [ ]:
# --- Create expanding layers using concatenation
concat = lambda a, b : layers.Concatenate()([a, b])

l6 =  tran2(48, conv1(48, l5))
l7 =  tran2(32, conv1(48, concat(convZ(48, 2, l4), l6)))
l8 =  tran2(16, conv1(32, concat(convZ(32, 3, l3), l7)))
l9 =  tran2(8,  conv1(16, concat(convZ(16, 3, l2), l8)))
l10 = conv1(8,  conv1(8,  concat(convZ(8,  3, l1), l9)))

In [ ]:
# --- Create logits

### Define model

In [ ]:
# --- Define model

# --- Create logits
logits = {}
logits['tumor'] = layers.Conv3D(
    name='tumor',
    filters=2, 
    strides=1, 
    kernel_size=(1, 3, 3), 
    padding='same',
    kernel_initializer='he_normal')(l10)
# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile and train model

In [ ]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={'tumor': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={'tumor': custom.dsc(cls=1)},
    experimental_run_tf_function=False)
# --- Train the model
model.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=500,
    validation_freq=4,
    use_multiprocessing=True)

Epoch 1/12
500/500 [==============================] - 378s 734ms/step - loss: 0.5051 - dsc_1: 0.2951
Epoch 2/12
500/500 [==============================] - 365s 730ms/step - loss: 0.0440 - dsc_1: 0.7699
Epoch 3/12
500/500 [==============================] - 365s 731ms/step - loss: 0.0200 - dsc_1: 0.8089
Epoch 4/12
500/500 [==============================] - 706s 1s/step - loss: 0.0141 - dsc_1: 0.8166 - val_loss: 0.0132 - val_dsc_1: 0.8304
Epoch 5/12
500/500 [==============================] - 363s 726ms/step - loss: 0.0112 - dsc_1: 0.8538
Epoch 6/12
500/500 [==============================] - 367s 733ms/step - loss: 0.0102 - dsc_1: 0.8367
Epoch 7/12
500/500 [==============================] - 364s 728ms/step - loss: 0.0091 - dsc_1: 0.8596
Epoch 8/12
500/500 [==============================] - 692s 1s/step - loss: 0.0083 - dsc_1: 0.8770 - val_loss: 0.0103 - val_dsc_1: 0.8523
Epoch 9/12
500/500 [==============================] - 347s 695ms/step - loss: 0.0078 - dsc_1: 0.8563
Epoch 10/12
500/500

In [ ]:
def dice(y_true, y_pred, c=1, epsilon=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      (np.ndarray) y_true : ground-truth label
      (np.ndarray) y_pred : predicted logits scores
      (int)             c : class to calculate DSC on
    
    """
    assert y_true.ndim == y_pred.ndim
    
    true = y_true[..., 0] == c
    pred = np.argmax(y_pred, axis=-1) == c 

    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred) + epsilon
    
    return A / B

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc_train_m2 = []
dsc_valid_m2=[]

for x, y in test_train:
    
    # --- Predict
    x['dat'] = np.pad(x['dat'], ((0, 0), (1, 1), (0, 0), (0, 0), (0, 0)))
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['tumor']

    # --- Argmax
    dsc_train_m2.append(dice(y['tumor'][0], logits[0], c=1))
    
dsc_train_m2 = np.array(dsc_train_m2)


for x, y in test_valid:
    
    # --- Predict
    x['dat'] = np.pad(x['dat'], ((0, 0), (1, 1), (0, 0), (0, 0), (0, 0)))
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['tumor']

    0# --- Argmax
    dsc_valid_m2.append(dice(y['tumor'][0], logits[0], c=1))
    
dsc_valid_m2 = np.array(dsc_valid_m2)

[ 2021-05-14 18:31:30 ] [====================] 100.000% : Iterating | 000074    

In [ ]:
model.summary()
df_train.to_csv('./train_results_m2.csv')
df_valid.to_csv('./valid_results_m2.csv')
model.save('./model_2.hdf5')

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, None, 240, 2 0                                            
__________________________________________________________________________________________________
conv3d_23 (Conv3D)              (None, None, 240, 24 296         dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization_27 (BatchNo (None, None, 240, 24 32          conv3d_23[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, None, 240, 24 0           batch_normalization_27[0][0]     
____________________________________________________________________________________________

In [ ]:
print(np.mean(dsc_train_m2), np.mean(dsc_valid_m2), 
      np.median(dsc_valid_m2), np.median(dsc_train_m2), 
      np.quantile(dsc_valid_m2, 0.75), np.quantile(dsc_train_m2, 0.75))



0.884958877754717 0.845669669432368 0.893690591820192 0.9077242954767188 0.9266174131859266 0.9326295618325435


## 3. Custom architecture

Finally, using any of the customizations described in class, find a top-performing model that yields some incremental benefit over the two baseline models above. Modifications that may be used include (but are not limited to):

* hybrid 3D/2D network
* residual connections
* added convolutions between contracting and expanding layers 
* modifications to the convolutional blocks including ResNet, Inception, SE-Net

In [ ]:
# --- Choose input (may copy the generator code from above)
# --- Input ==> 3 x 240 x 240 x 4
configs = {
    'specs': {
        'xs': {'dat': {'shape': [3, 240, 240, 4]}},
        'ys': {'tumor': {'norms': {'clip': {'max': 1}}}}}}

gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-mip', keyword='mip*vox', configs=configs)
inputs = client.get_inputs(Input)

In [ ]:
# --- Define 2D conv (xy-features)
conv_2d = lambda x, filters, strides : layers.Conv3D(
    filters=filters, 
    strides=strides, 
    kernel_size=(1, 3, 3), 
    padding='same',
    kernel_initializer='he_normal')(x)

# --- Define 1D conv (z-features)
conv_1d = lambda x, filters, k=2 : layers.Conv3D(
    filters=filters,
    strides=1,
    kernel_size=(k, 1, 1),
    padding='valid',
    kernel_initializer='he_normal')(x)

    # --- Define lambda functions
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)

# --- Define stride-1 3D, stride-2 3D and stride-1 1D (z-subsample) blocks
conv1 = lambda filters, x : relu(norm(conv_2d(x, filters, strides=(1, 1, 1))))
conv2 = lambda filters, x : relu(norm(conv_2d(x, filters, strides=(1, 2, 2))))
convZ = lambda filters, k, x : relu(norm(conv_1d(x, filters, k=k)))

# --- Define arbitrary input
dat = Input(shape=(3, 240, 240, 4))

# --- Define contracting layers
l1 = conv1(8,  inputs['dat'])

# --- Squeeze (global pool)
p1 = layers.GlobalAveragePooling3D()(l1)

# --- Excitation (reduce channels to 1 / R) ==> in this example set R = 4 arbitrarily
ch = int(p1.shape[-1] / 4)
f1 = layers.Dense(ch, activation='relu')(p1)

# --- Scale (expand channels to original size)
scale = layers.Dense(l1.shape[-1], activation='sigmoid')(f1)
scale = layers.Reshape((1, 1, 1, l1.shape[-1]))(scale)    

# --- Modify l1
l1 = l1 * scale

l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, convZ(48, 2, conv2(48, l3)))
l5 = conv1(64, convZ(64, 2, conv2(64, l4)))


Model(inputs=inputs, outputs=l5)({'dat': dat})

# --- 3-slices to 1-slice
p3 = convZ(32, 3, l3)
print(Model(inputs=inputs, outputs=l3)({'dat': dat}).shape)
print(Model(inputs=inputs, outputs=p3)({'dat': dat}).shape)

# --- 2-slices to 1-slice
p4 = convZ(48, 2, l4)
print(Model(inputs=inputs, outputs=l4)({'dat': dat}).shape)
print(Model(inputs=inputs, outputs=p4)({'dat': dat}).shape)
# --- Define 2D transpose

tran = lambda x, filters : layers.Conv3DTranspose(
    filters=filters, 
    strides=(1, 2, 2),
    kernel_size=(1, 3, 3),
    padding='same',
    kernel_initializer='he_normal')(x)

# --- Define transpose block
tran2 = lambda filters, x : relu(norm(tran(x, filters)))

# --- Create expanding layers using concatenation
concat = lambda a, b : layers.Concatenate()([a, b])

l6 =  tran2(48, conv1(48, l5))
l7 =  tran2(32, conv1(48, concat(convZ(48, 2, l4), l6)))
l8 =  tran2(16, conv1(32, concat(convZ(32, 3, l3), l7)))
l9 =  tran2(8,  conv1(16, concat(convZ(16, 3, l2), l8)))
l10 = conv1(8,  conv1(8,  concat(convZ(8,  3, l1), l9)))

(None, 3, 60, 60, 32)
(None, 1, 60, 60, 32)
(None, 2, 30, 30, 48)
(None, 1, 30, 30, 48)


### Define model

In [ ]:
# --- Define model

# --- Create logits
logits = {}
logits['tumor'] = layers.Conv3D(
    name='tumor',
    filters=2, 
    strides=1, 
    kernel_size=(1, 3, 3), 
    padding='same',
    kernel_initializer='he_normal')(l10)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile and train model

In [31]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={'tumor': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={'tumor': custom.dsc(cls=1)},
    experimental_run_tf_function=False)
# --- Train the model
model.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=500,
    validation_freq=4,
    use_multiprocessing=True)


Epoch 1/12
500/500 [==============================] - 370s 726ms/step - loss: 0.2414 - dsc_1: 0.3876
Epoch 2/12
500/500 [==============================] - 364s 728ms/step - loss: 0.0250 - dsc_1: 0.7784
Epoch 3/12
500/500 [==============================] - 360s 720ms/step - loss: 0.0142 - dsc_1: 0.8304
Epoch 4/12
500/500 [==============================] - 707s 1s/step - loss: 0.0113 - dsc_1: 0.8541 - val_loss: 0.0113 - val_dsc_1: 0.8320
Epoch 5/12
500/500 [==============================] - 363s 727ms/step - loss: 0.0094 - dsc_1: 0.8537
Epoch 6/12
500/500 [==============================] - 367s 733ms/step - loss: 0.0093 - dsc_1: 0.8668
Epoch 7/12
500/500 [==============================] - 364s 728ms/step - loss: 0.0083 - dsc_1: 0.8543
Epoch 8/12
500/500 [==============================] - 708s 1s/step - loss: 0.0078 - dsc_1: 0.8588 - val_loss: 0.0097 - val_dsc_1: 0.8455
Epoch 9/12
500/500 [==============================] - 364s 730ms/step - loss: 0.0077 - dsc_1: 0.8715
Epoch 10/12
500/500

In [32]:
def dice(y_true, y_pred, c=1, epsilon=1):
    """````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      (np.ndarray) y_true : ground-truth label
      (np.ndarray) y_pred : predicted logits scores
      (int)             c : class to calculate DSC on
    
    """
    assert y_true.ndim == y_pred.ndim
    
    true = y_true[..., 0] == c
    pred = np.argmax(y_pred, axis=-1) == c `````````````````````````````````````````````````````````````````````````````````````````````````````````````````````

    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred) + epsilon
    
    return A / B

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* Dice score, mean
* Dice score, median
* Dice score, 25th percentile
* Dice score, 75th percentile

As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the Dice score values on the entire 3D volume (not slice-by-slice).

### Performance

The following minimum performance metrics must be met for full credit:

1. **2D U-Net**: median Dice score > 0.64
2. **Hybrid 3D/2D U-Net**: median Dice score > 0.68
3. **Custom architecture**: median Dice score > 0.72

In [33]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc_train_m3 = []
dsc_valid_m3=[]

for x, y in test_train:
    
    # --- Predict
    x['dat'] = np.pad(x['dat'], ((0, 0), (1, 1), (0, 0), (0, 0), (0, 0)))
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['tumor']

    # --- Argmax
    dsc_train_m3.append(dice(y['tumor'][0], logits[0], c=1))
    
dsc_train_m3 = np.array(dsc_train_m3)


for x, y in test_valid:
    
    # --- Predict
    x['dat'] = np.pad(x['dat'], ((0, 0), (1, 1), (0, 0), (0, 0), (0, 0)))
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['tumor']

    # --- Argmax
    dsc_valid_m3.append(dice(y['tumor'][0], logits[0], c=1))
    
dsc_valid_m3 = np.array(dsc_valid_m3)

[ 2021-05-14 20:11:03 ] [====================] 100.000% : Iterating | 000074    

In [ ]:
print(np.mean(dsc_train_m3), np.mean(dsc_valid_m3), 
      np.median(dsc_valid_m3), np.median(dsc_train_m3), 
      np.quantile(dsc_valid_m3, 0.75), np.quantile(dsc_train_m3, 0.75))

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the three different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          TRAINING                                VALIDATION
          mean | median | 25th-tile | 75th-tile | mean | median | 25th-tile | 75th-tile
model 1
model 2
model 3
```

As above, statistics for both training and validation should be provided.

In [35]:
# --- Create *.csv
df_results = pd.DataFrame(columns=['train mean', 'train median', 'train 25th percentile', 'train 75th percentiele', 'valid mean', 'valid median', 'valid 25th percentile', 'valid 75th percentiele'])
df_results.loc[0]=np.mean(dsc_train), np.median(dsc_train), np.quantile(dsc_train, 0.25), np.quantile(dsc_train, 0.75), np.mean(dsc_valid), np.median(dsc_valid), np.quantile(dsc_valid, 0.25), np.quantile(dsc_valid, 0.75)
df_results.loc[1]=np.mean(dsc_train_m2), np.median(dsc_train_m2), np.quantile(dsc_train_m2, 0.25), np.quantile(dsc_train_m2, 0.75), np.mean(dsc_valid_m2), np.median(dsc_valid_m2), np.quantile(dsc_valid_m2, 0.25), np.quantile(dsc_valid_m2, 0.75)
df_results.loc[2]=np.mean(dsc_train_m3), np.median(dsc_train_m3), np.quantile(dsc_train_m3, 0.25), np.quantile(dsc_train_m3, 0.75), np.mean(dsc_valid_m3), np.median(dsc_valid_m3), np.quantile(dsc_valid_m3, 0.25), np.quantile(dsc_valid_m3, 0.75)
                              
# --- Serialize *.csv
fname = './results.csv'
df_results.to_csv(fname)
df_train.to_csv('./train_results_m3.csv')
df_valid.to_csv('./valid_results_m3.csv')
model.save('./model_3.hdf5')

# Summary

In addition to algorithm training as above, a 1-2 page write-up is required for this project. The goal is to *briefly* summarize algorithm design and key results. The write-up should be divided into three sections: methods; results; discussion. More detailed information and tips can be found here: https://github.com/peterchang77/dl_tutor/blob/master/cs190/spring_2021/notebooks/midterm/checklist.md.

### Methods

In this section, include details such as:

* **Data**: How much data was used. How many cases were utilized for training and validation?
* **Network design**: What are the different network architectures? How many layers and parameters? Were 2D or 3D operations used? Recall that the `model.summary(...)` can be used to provide key summary statistics for this purpose. If desired, feel free to include a model figure or diagram.
* **Implementation**: How was training implemented. What are the key hyperparameters (e.g. learning rate, batch size, optimizer, etc)? How many training iterations were required for convergence? Did these hyperparameters change during the course of training?
* **Statistics**: What statistics do you plan to use to evaluate model accuracy? 

### Results

In this section, briefly summarize experimental results (a few sentences), and include the result table(s) as derived above.

### Discussion

Were the results expected or unexpected? What accounts for the differences in performance between the algorithms? How did you choose the network architecture implemented in your final model? Feel free to elaborate on any additional observations noted during the course of this expierment.

# Submission


### Canvas

Once you have completed the midterm assignment, download the necessary files from Google Colab and your Google Drive. As in prior assigments, be sure to prepare:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv` (compiled for all three parts)
* final (trained) model: `[UCInetID]_model.hdf5` (three separate files for all three parts)

In addition, submit the summary write-up as in any common document format (`.docx`, `.tex`, `.pdf`, etc):

* final summary write-up: `[UCInetID]_summary.[docx|tex|pdf]`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadsheet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.